In [13]:
#Import all packages we’ll need.
import json
import numpy as np
import random
import nltk
import utils as u
import pickle
nltk.download('punkt')
nltk.download('wordnet')
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.models import load_model
import tkinter
from tkinter import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
class ChatModel:
    def __init__(self):
        #Call tokenizing procedure
        w, words, documents, classes, self._intents = self.tokenizing('intents.json')
        #Call lemmatizing procedure
        w, words, documents, classes, lemmatizer = self.lemmatizing(w, words, documents, classes)
        #Call training_data procedure
        self._train_x, self._train_y = self.training_data(w, words, documents, classes, lemmatizer)
        #Call tokenizing procedure
        self._model = self.training(self._train_x, self._train_y)
        
    def tokenizing(self,url):
        words=[]
        classes = []
        documents = []
        intents = json.loads(open(url).read())

        for intent in intents['intents']:
            for pattern in intent['patterns']:
                #tokenize each word
                w = nltk.word_tokenize(pattern)
                words.extend(w)
                #add documents in the corpus
                documents.append((w, intent['tag']))
                # add to our classes list
                if intent['tag'] not in classes:
                    classes.append(intent['tag'])

        return w, words, documents, classes, intents

    def lemmatizing(self, w, words, documents, classes):
        ignore_words = ['?', '!']
        lemmatizer = nltk.stem.WordNetLemmatizer()

        # lemmatize, lower each word and remove duplicates
        words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

        # sort classes and words
        classes = sorted(list(set(classes)))
        words = sorted(list(set(words)))
        # documents = combination between patterns and intents
        print (len(documents), "documents")

        # classes = intents
        print (len(classes), "classes", classes)

        # words = all words, vocabulary
        print (len(words), "unique lemmatized words", words)

        u.create_pickle(words, 'pickles\words.pkl') 
        u.create_pickle(classes, 'pickles\classes.pkl')
        return w, words, documents, classes, lemmatizer

    def training_data(self, w, words, documents, classes, lemmatizer):
        # create our training data
        training = []
        train_x = []
        train_y = []
        # create an empty array for our output
        output_empty = [0] * len(classes)

        # training set, bag of words for each sentence
        for doc in documents:
            # initialize our bag of words
            bag = []
            # list of tokenized words for the pattern
            pattern_words = doc[0]
            # lemmatize each word - create base word, in attempt to represent related words
            pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
            # create our bag of words array with 1, if word match found in current pattern

            for w in words:
                bag.append(1) if w in pattern_words else bag.append(0)

            # output is a '0' for each tag and '1' for current tag (for each pattern)
            output_row = list(output_empty)
            output_row[classes.index(doc[1])] = 1
            training.append([bag, output_row])

        # shuffle our features and turn into np.array
        random.shuffle(training)
        training = np.array(training)
        # create train and test lists. X - patterns, Y - intents
        train_x = list(training[:,0])
        train_y = list(training[:,1])

        print("Training data created")
        return train_x, train_y

    def training(self,train_x, train_y):
        #Sequential from Keras
        # Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
        # equal to number of intents to predict output intent with softmax
        model = Sequential()
        model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(len(train_y[0]), activation='softmax'))
        # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        #fitting and saving the model 
        hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
        model.save('chatbot_model.h5', hist)
        print("modeseql created")

        return model
    
    def get_train_x(self):
        return self._train_x
    def get_train_y(self):
        return self._train_y
    def get_model(self):
        return self._model
    def get_intents(self):
        return self._intents

### Load and Preprocess Data

In [18]:
class ChatApp:

    def __init__(self):
        self.cM = ChatModel()
        self._lemmatizer = nltk.stem.WordNetLemmatizer()
        self._model = load_model('chatbot_model.h5')
        self._intents = self.cM.get_intents() 
        self._words = u.load_pickle('pickles\words.pkl')
        self._classes = u.load_pickle('pickles\classes.pkl')
        
    def clean_up_sentence(self,sentence):
        # tokenize the pattern - split words into array
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word - create short form for word
        sentence_words = [self._lemmatizer.lemmatize(word.lower()) for word in sentence_words]
        return sentence_words

    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
    def bow(self, sentence, words, show_details=True):
        # tokenize the pattern
        sentence_words = self.clean_up_sentence(sentence)
        # bag of words - matrix of N words, vocabulary matrix
        bag = [0]*len(words) 
        for s in sentence_words:
            for i,w in enumerate(words):
                if w == s: 
                    # assign 1 if current word is in the vocabulary position
                    bag[i] = 1
                    if show_details:
                        print ("found in bag: %s" % w)
        return(np.array(bag))

    def predict_class(self, sentence, model):
        ERROR_THRESHOLD = 0.25
        # filter out predictions below a threshold
        p = self.bow(sentence, self._words, show_details=False)
        res = self._model.predict(np.array([p]))[0]
        
        results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
        # sort by strength of probability
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({"intent": self._classes[r[0]], "probability": str(r[1])})
        return return_list


    def getResponse(self, ints, intents_json):
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if(i['tag']== tag):
                result = random.choice(i['responses'])
                break
        return result

    def chatbot_response(self, text):
        ints = self.predict_class(text, self._model)
        res = self.getResponse(ints, self._intents)
        return res

In [19]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "You: " + msg + '\n\n')
            ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
            res = ChatApp().chatbot_response(msg)
            ChatLog.insert(END, "Bot: " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)


base = Tk()
base.title("ChatBot - SL")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']
Training data created
Epoch 1/200
47

47/47 [==============================] - 0s 275us/step - loss: 0.0529 - accuracy: 0.9787
Epoch 72/200
47/47 [==============================] - 0s 254us/step - loss: 0.0964 - accuracy: 0.9574
Epoch 73/200
47/47 [==============================] - 0s 268us/step - loss: 0.0621 - accuracy: 0.9787
Epoch 74/200
47/47 [==============================] - 0s 276us/step - loss: 0.0393 - accuracy: 1.0000
Epoch 75/200
47/47 [==============================] - 0s 243us/step - loss: 0.0327 - accuracy: 1.0000
Epoch 76/200
47/47 [==============================] - 0s 279us/step - loss: 0.0511 - accuracy: 0.9787
Epoch 77/200
47/47 [==============================] - 0s 255us/step - loss: 0.0321 - accuracy: 1.0000
Epoch 78/200
47/47 [==============================] - 0s 298us/step - loss: 0.0432 - accuracy: 1.0000
Epoch 79/200
47/47 [==============================] - 0s 318us/step - loss: 0.0967 - accuracy: 0.9574
Epoch 80/200
47/47 [==============================] - 0s 318us/step - loss: 0.0315 - accuracy: 

47/47 [==============================] - 0s 233us/step - loss: 0.0330 - accuracy: 1.0000
Epoch 151/200
47/47 [==============================] - 0s 254us/step - loss: 0.0047 - accuracy: 1.0000
Epoch 152/200
47/47 [==============================] - 0s 255us/step - loss: 0.0296 - accuracy: 1.0000
Epoch 153/200
47/47 [==============================] - 0s 234us/step - loss: 0.0284 - accuracy: 1.0000
Epoch 154/200
47/47 [==============================] - 0s 276us/step - loss: 0.0316 - accuracy: 1.0000
Epoch 155/200
47/47 [==============================] - 0s 255us/step - loss: 0.0249 - accuracy: 1.0000
Epoch 156/200
47/47 [==============================] - 0s 233us/step - loss: 0.0776 - accuracy: 0.9787
Epoch 157/200
47/47 [==============================] - 0s 233us/step - loss: 0.0119 - accuracy: 1.0000
Epoch 158/200
47/47 [==============================] - 0s 276us/step - loss: 0.0211 - accuracy: 1.0000
Epoch 159/200
47/47 [==============================] - 0s 277us/step - loss: 0.0462 - a

47/47 [==============================] - 0s 276us/step - loss: 0.5507 - accuracy: 0.8298
Epoch 21/200
47/47 [==============================] - 0s 255us/step - loss: 0.6941 - accuracy: 0.8085
Epoch 22/200
47/47 [==============================] - 0s 233us/step - loss: 0.5801 - accuracy: 0.8511
Epoch 23/200
47/47 [==============================] - 0s 255us/step - loss: 0.5387 - accuracy: 0.8511
Epoch 24/200
47/47 [==============================] - 0s 233us/step - loss: 0.4724 - accuracy: 0.8511
Epoch 25/200
47/47 [==============================] - 0s 211us/step - loss: 0.3992 - accuracy: 0.9149
Epoch 26/200
47/47 [==============================] - 0s 211us/step - loss: 0.4305 - accuracy: 0.8298
Epoch 27/200
47/47 [==============================] - 0s 211us/step - loss: 0.3196 - accuracy: 0.9574
Epoch 28/200
47/47 [==============================] - 0s 276us/step - loss: 0.3926 - accuracy: 0.8936
Epoch 29/200
47/47 [==============================] - 0s 276us/step - loss: 0.3550 - accuracy: 

47/47 [==============================] - 0s 297us/step - loss: 0.0583 - accuracy: 0.9787
Epoch 101/200
47/47 [==============================] - 0s 297us/step - loss: 0.0441 - accuracy: 1.0000
Epoch 102/200
47/47 [==============================] - 0s 297us/step - loss: 0.0355 - accuracy: 1.0000
Epoch 103/200
47/47 [==============================] - 0s 276us/step - loss: 0.1147 - accuracy: 0.9787
Epoch 104/200
47/47 [==============================] - ETA: 0s - loss: 0.0287 - accuracy: 1.00 - 0s 297us/step - loss: 0.0407 - accuracy: 1.0000
Epoch 105/200
47/47 [==============================] - 0s 297us/step - loss: 0.0532 - accuracy: 1.0000
Epoch 106/200
47/47 [==============================] - 0s 276us/step - loss: 0.0274 - accuracy: 1.0000
Epoch 107/200
47/47 [==============================] - 0s 276us/step - loss: 0.0443 - accuracy: 1.0000
Epoch 108/200
47/47 [==============================] - 0s 276us/step - loss: 0.0314 - accuracy: 1.0000
Epoch 109/200
47/47 [========================

...

47/47 [==============================] - 0s 297us/step - loss: 0.3458 - accuracy: 0.9362
Epoch 31/200
47/47 [==============================] - 0s 340us/step - loss: 0.2404 - accuracy: 0.9574
Epoch 32/200
47/47 [==============================] - 0s 318us/step - loss: 0.2829 - accuracy: 0.8936
Epoch 33/200
47/47 [==============================] - 0s 318us/step - loss: 0.2445 - accuracy: 0.9149
Epoch 34/200
47/47 [==============================] - 0s 318us/step - loss: 0.2186 - accuracy: 0.9362
Epoch 35/200
47/47 [==============================] - 0s 318us/step - loss: 0.1624 - accuracy: 0.9787
Epoch 36/200
47/47 [==============================] - 0s 297us/step - loss: 0.2027 - accuracy: 0.9362
Epoch 37/200
47/47 [==============================] - 0s 297us/step - loss: 0.1104 - accuracy: 0.9787
Epoch 38/200
47/47 [==============================] - 0s 233us/step - loss: 0.1553 - accuracy: 1.0000
Epoch 39/200
47/47 [==============================] - 0s 255us/step - loss: 0.1237 - accuracy: 

47/47 [==============================] - 0s 297us/step - loss: 0.0492 - accuracy: 0.9787
Epoch 111/200
47/47 [==============================] - 0s 255us/step - loss: 0.0178 - accuracy: 1.0000
Epoch 112/200
47/47 [==============================] - 0s 318us/step - loss: 0.0375 - accuracy: 1.0000
Epoch 113/200
47/47 [==============================] - 0s 233us/step - loss: 0.0270 - accuracy: 1.0000
Epoch 114/200
47/47 [==============================] - 0s 233us/step - loss: 0.0310 - accuracy: 1.0000
Epoch 115/200
47/47 [==============================] - 0s 276us/step - loss: 0.0644 - accuracy: 0.9787
Epoch 116/200
47/47 [==============================] - 0s 255us/step - loss: 0.0570 - accuracy: 0.9787
Epoch 117/200
47/47 [==============================] - 0s 233us/step - loss: 0.0289 - accuracy: 1.0000
Epoch 118/200
47/47 [==============================] - 0s 276us/step - loss: 0.0070 - accuracy: 1.0000
Epoch 119/200
47/47 [==============================] - 0s 213us/step - loss: 0.1222 - a

47/47 [==============================] - 0s 277us/step - loss: 0.0168 - accuracy: 1.0000
Epoch 189/200
47/47 [==============================] - 0s 276us/step - loss: 0.0042 - accuracy: 1.0000
Epoch 190/200
47/47 [==============================] - 0s 234us/step - loss: 0.0262 - accuracy: 1.0000
Epoch 191/200
47/47 [==============================] - 0s 276us/step - loss: 0.0084 - accuracy: 1.0000
Epoch 192/200
47/47 [==============================] - 0s 297us/step - loss: 0.0377 - accuracy: 0.9787
Epoch 193/200
47/47 [==============================] - 0s 296us/step - loss: 0.0183 - accuracy: 1.0000
Epoch 194/200
47/47 [==============================] - 0s 297us/step - loss: 0.0056 - accuracy: 1.0000
Epoch 195/200
47/47 [==============================] - 0s 297us/step - loss: 0.0317 - accuracy: 1.0000
Epoch 196/200
47/47 [==============================] - 0s 283us/step - loss: 0.0037 - accuracy: 1.0000
Epoch 197/200
47/47 [==============================] - 0s 213us/step - loss: 0.0190 - a

47/47 [==============================] - 0s 276us/step - loss: 0.1475 - accuracy: 0.9362
Epoch 59/200
47/47 [==============================] - 0s 297us/step - loss: 0.0941 - accuracy: 0.9787
Epoch 60/200
47/47 [==============================] - 0s 277us/step - loss: 0.0926 - accuracy: 0.9787
Epoch 61/200
47/47 [==============================] - 0s 233us/step - loss: 0.1207 - accuracy: 0.9787
Epoch 62/200
47/47 [==============================] - 0s 255us/step - loss: 0.2104 - accuracy: 0.9787
Epoch 63/200
47/47 [==============================] - 0s 255us/step - loss: 0.0663 - accuracy: 1.0000
Epoch 64/200
47/47 [==============================] - 0s 254us/step - loss: 0.1343 - accuracy: 0.9574
Epoch 65/200
47/47 [==============================] - 0s 276us/step - loss: 0.0732 - accuracy: 0.9787
Epoch 66/200
47/47 [==============================] - 0s 233us/step - loss: 0.0396 - accuracy: 1.0000
Epoch 67/200
47/47 [==============================] - 0s 233us/step - loss: 0.0603 - accuracy: 

47/47 [==============================] - 0s 297us/step - loss: 0.0113 - accuracy: 1.0000
Epoch 139/200
47/47 [==============================] - 0s 276us/step - loss: 0.0173 - accuracy: 1.0000
Epoch 140/200
47/47 [==============================] - 0s 297us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 141/200
47/47 [==============================] - 0s 276us/step - loss: 0.0432 - accuracy: 0.9787
Epoch 142/200
47/47 [==============================] - 0s 276us/step - loss: 0.0119 - accuracy: 1.0000
Epoch 143/200
47/47 [==============================] - 0s 276us/step - loss: 0.0350 - accuracy: 1.0000
Epoch 144/200
47/47 [==============================] - 0s 254us/step - loss: 0.0437 - accuracy: 0.9787
Epoch 145/200
47/47 [==============================] - 0s 233us/step - loss: 0.0304 - accuracy: 1.0000
Epoch 146/200
47/47 [==============================] - 0s 255us/step - loss: 0.0138 - accuracy: 1.0000
Epoch 147/200
47/47 [==============================] - 0s 255us/step - loss: 0.0089 - a